In [1]:
import pandas as pd
from unicodedata import normalize

In [147]:
# OpLexicon https://www.inf.pucrs.br/linatural/wordpress/recursos-e-ferramentas/oplexicon/
    #https://www.researchgate.net/publication/262175717_Sentiment_Analysis_on_Twitter_Data_for_Portuguese_Language
    #https://aclanthology.org/W11-4507.pdf
lexico = pd.read_csv('dictionaries/lexico_v3.0.txt', sep=",", names=['palavra', 'tipo', 'score', 'ex'])

In [81]:
# Sentilex-PT http://b2find.eudat.eu/dataset/b6bd16c2-a8ab-598f-be41-1e7aeecd60d3
    # https://www.researchgate.net/publication/344644990_SentiLex-PT_Principais_caracteristicas_e_potencialidades
sentilex_lem = open('dictionaries/SentiLex-lem-PT02.txt','r')
sentilex_flex = open('dictionaries/SentiLex-flex-PT02.txt','r')

In [11]:
lexico.head()

,palavra,tipo,score,ex
0,=[,emot,-1,A
1,=@,emot,-1,A
2,=p,emot,-1,A
3,=P,emot,-1,A
4,=x,emot,-1,A


In [42]:
columns = [
    'word',
    'type',
    'score'
]

joint_dics = pd.DataFrame(columns=columns)

In [43]:
for row in lexico.itertuples(index=True, name='Pandas'):
    joint_dics = joint_dics.append({'word':getattr(row, 'palavra'),
                                    'type':getattr(row, 'tipo'), 
                                    'score':getattr(row, 'score')}, ignore_index=True)

In [44]:
joint_dics.head()

,word,type,score
0,=[,emot,-1
1,=@,emot,-1
2,=p,emot,-1
3,=P,emot,-1
4,=x,emot,-1


In [76]:
for i in sentilex_flex.readlines():
    pos_ponto = i.find('.')            # obtem a posiçãodo caracter ponto
    palavra = (i[:pos_ponto])          # Pega a palavra
    pos_palavra = palavra.find(',')    # obtem  a posição do caracter virgula
    palavra1 = (palavra[pos_palavra+1:])          # Pega a palavra
    palavra2 = (palavra[:pos_palavra])          # Pega a palavra
    pol_pos = i.find('POL')            # obtem a posição do inicio da string POL
    polaridade = (i[pol_pos+7:pol_pos+9]).replace(';','')         # obtem a polaridade da palavra
    pos = i.find(';')              # obtem posição do ponto e virgula
    tipo = (i[:pos])               
    pos = tipo.find('PoS')        
    tipo = tipo[pos+4:]         #pega o tipo              
    joint_dics = joint_dics.append({'word':palavra1,            
                                    'type':tipo, 
                                    'score':polaridade},
                                   ignore_index=True) # atualiza o dicionario com a palavra a polaridade
    if(palavra1!=palavra2):
         joint_dics = joint_dics.append({'word':palavra2,
                                    'type':tipo, 
                                    'score':polaridade},
                                   ignore_index=True) # atualiza o dicionario com a palavra a polaridade

In [82]:
for i in sentilex_lem.readlines():
    pos_ponto = i.find('.')            # obtem a posiçãodo caracter ponto
    palavra = (i[:pos_ponto])          # Pega a palavra
    pol_pos = i.find('POL')            # obtem a posição do inicio da string POL
    polaridade = (i[pol_pos+7:pol_pos+9]).replace(';','')         # obtem a polaridade da palavra
    pos = i.find(';')              # obtem posição do ponto e virgula
    tipo = (i[:pos])               
    pos = tipo.find('PoS')
    tipo = tipo[pos+4:]         #pega o tipo  
    joint_dics = joint_dics.append({'word':palavra,            
                                    'type':tipo, 
                                    'score':polaridade},
                                   ignore_index=True) # atualiza o dicionario com a palavra a polaridade

In [96]:
joint_dics.to_csv(r'dictionaries/joint_dictionaties.csv', index = False)

In [93]:
joint_dics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203988 entries, 0 to 203987
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   word    203988 non-null  object
 1   type    203988 non-null  object
 2   score   203988 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.7+ MB


In [92]:
joint_dics['score'] = pd.to_numeric(joint_dics['score'])

In [282]:
data = pd.read_csv("tweets_processed.csv")

In [283]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1000 non-null   int64  
 1   text          1000 non-null   object 
 2   preprocessed  1000 non-null   object 
 3   sentiment     0 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.4+ KB


In [131]:
joint_dics_backup = joint_dics

In [133]:
joint_dics['type'] = joint_dics['type'].str.upper()

/home/claesia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [134]:
joint_dics['word'] = joint_dics['word'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

/home/claesia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [138]:
joint_dics = joint_dics.drop_duplicates()

In [139]:
joint_dics.loc[joint_dics['word']=='odio']

,word,type,score
155285,odio,N,-1


In [140]:
joint_dics.loc[joint_dics['word']=='abusivo']

,word,type,score
900,abusivo,ADJ,-1


In [143]:
joint_dics.loc[joint_dics['word']=='alegria']

,word,type,score
40097,alegria,N,1


In [145]:
joint_dics.loc[joint_dics['word']=='viajar']

,word,type,score
31752,viajar,VB,0


In [146]:
joint_dics.to_csv(r'dictionaries/joint_dictionaties_processed.csv', index = False)

In [249]:
joint_dics.loc[joint_dics['word']=='odeio'].score.sum()

0

In [258]:
def score_sentiment(text):
    text = text.lower()
    l_sentimento = []
    for word in text.split():
        if not joint_dics.loc[joint_dics['word']==word].empty:
            l_sentimento.append(int(joint_dics.loc[joint_dics['word']==word].score.sum()))
    score = sum(l_sentimento)
    return score

In [259]:
score_sentiment('eu odeio dias frios') # I hate cold days

-1

In [260]:
score_sentiment('hoje é um dia especial') # today is a special day

1

In [284]:
def classify():
    for index, row in enumerate(data['text']):
        data.at[index, 'sentiment'] =  score_sentiment(data.at[index, 'preprocessed'])

In [285]:
classify()

In [286]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1000 non-null   int64  
 1   text          1000 non-null   object 
 2   preprocessed  1000 non-null   object 
 3   sentiment     1000 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.4+ KB


In [303]:
data.head()

,id,text,preprocessed,sentiment
0,1435695806452011008,@projetoskinny não vai com nenhum pq a gente t...,não vai com nenhum gente numa pandemia,0
1,1435695762948726787,@renanmouraglobo Em SP governo só libera em SP...,governo libera logo essa acordo furado espera ...,-1
2,1435695755805741059,viagem nunca q ia dar certo no meio da pandemi...,viagem nunca dar certo meio pandemia queria festa,0
3,1435695709446197252,@FBITricolor até porque os hospitais estão vaz...,até porque hospitais estão vazios pandemia con...,0
4,1435695680861921280,also foi bem antes da pandemia (acho q o tweet...,also foi bem antes pandemia acho tweet referia...,0


In [302]:
data['sentiment'] = data['sentiment'].astype(int) 

In [304]:
data.to_csv(r'tweets_classified.csv', index = False)

In [305]:
def count_sentiment():
    i = 0
    positivo = 0
    negativo = 0
    neutro = 0
    for index, row in enumerate(data['text']):
        if data.at[index, 'sentiment'] == 0:
            neutro+=1
        elif data.at[index, 'sentiment'] > 0:
            positivo+=1
        else:
            negativo+=1
            
    print("Positivo: ", positivo)
    print("Negativo: ", negativo)
    print("Neutro: ", neutro)

In [306]:
count_sentiment()

Positivo:  305
Negativo:  418
Neutro:  277


In [308]:
data.loc[data['sentiment']>0][['text', 'sentiment']].style.set_properties(subset=['text'], **{'width': '300px'})

,text,sentiment
6,"Eu sou a pessoa mais feliz do mundo por ter acompanhado essa história desde o começo. Sen Çal Kapımı, obrigada por salvar minha sanidade no meio dessa pandemia 🥺🤧 Vou sentir saudades todos os dias ♥️ https://t.co/4gX6N3rhSs",1
8,"Bolsonarismo disposto a derrubar... Bolsonaro. Fazer lockout em favor do governo, é a primeira vez na história do mundo. O presidente não era contra iniciativas sanitárias que, para combater a pandemia, dificultavam a circulação? Pra ""defender o governo"", aí pode? https://t.co/U7Df1gyGlV",2
9,Tudo der certo: segunda dose no braço e se a pandemia não estourar dnv pelo amor de deus preciso trabalhar,3
21,Lembrando aqui da vez que passei o dia todinho de pandemia jogando tênis em GTA V 🤭🤭🤭 eu era bem melhor atropelando o povo pelas ruas e ultrapassando os sinais vermelhos,1
23,"@serepjms @itachionce @emoktaehy pare! sou uma pessoa mudada agr, a pandemia me transformou em santa JURO",1
25,"“O juiz negacionista” que boa forma de tentarem humilhar alguém que, mesmo com arrogância, defendeu algo que me parece inerente. Um dos poucos agentes de autoridade que não se quer aproveitar da pandemia para andar aí a distribuir porrada e a controlar as pessoas.",2
28,Compartilhe https://t.co/qtJDW2oOiq DêLike Dicas para Realizar Reuniões Virtuais durante a Pandemia ✔ Brazil SFE Book® #AndréLuizBernardes #eBook #REUNIÕESONLINE #REUNIÃOVIRTUAL #REUNIÕESREMOTAS #Vídeoconferência #Webconferência #Skype #WhatsApp #GoolgleMeet #Zoom #Webex https://t.co/Lz6AJA1N4S,1
35,"Dou por declarado o fim da Pandemia! Estarei enviando por e-mail o certificado com horas pra ACC e terá extra para aqueles que passaram a Pandemia curtindo a vida como se nada estivesse acontecendo e ainda haverá outro extra para aqueles que continuarem, um cupom para a funerária",3
38,"@nemligaguria_ eu sempre junto isso com não gostar de comemorar. (ano passado eu estava cogitando, ai veio a pandemia... esse ano voltei a ser eu)",1
41,Sonho pós pandemia: https://t.co/Jm9r9uL7T6,1


In [309]:
data.loc[data['sentiment']==0][['text', 'sentiment']].style.set_properties(subset=['text'], **{'width': '300px'})

,text,sentiment
0,@projetoskinny não vai com nenhum pq a gente tá numa pandemia .,0
2,viagem nunca q ia dar certo no meio da pandemia e eu já n queria festa,0
3,@FBITricolor até porque os hospitais estão vazios e a pandemia controlada graças a deus,0
4,also foi bem antes da pandemia (acho q o tweet se referia a isso),0
7,"@beccacvargas naooo, mais ou menos no 4º mes de pandemia eu surtei e voltei a comer carne. só não sei se vou manter kkkk",0
16,"Minha irmã de outra mãe a thread; Oi gente, hoje vou expor uma parte minha na internet sobre um assunto que pra mim é muito delicado, mas tomei coragem para me abrir sobre e aceitar quem eu sou. De um tempo pra cá a pandemia fudeu muito meu psicológico e eu fiquei muito mal e + https://t.co/dprOQD4xzT",0
17,"@eocoreinha @HaddadDebochado Só para não dizer, que não falei das pérolas do fique em casa https://t.co/3o0BixASdY",0
18,vcs realmente não querem que a pandemia acabe https://t.co/PU33KdaqMf,0
19,e eu achava que a pandemia tava acabando… https://t.co/tlzNh8j2oA,0
20,"Então, @intercom_2020, justamente pela dificuldade da pandemia é q a gte esperava mais. Tem todo tipo de transporte por aí, de coisas bem mais pesadas e/ou quebráveis que os troféus e elas chegam em perfeito estado em todo o Brasil. Boton é brinde! Tristeza e decepção. https://t.co/9I3R2K3LeL",0


In [310]:
data.loc[data['sentiment']<0][['text', 'sentiment']].style.set_properties(subset=['text'], **{'width': '300px'})

,text,sentiment
1,"@renanmouraglobo Em SP governo só libera em SP, logo essa já é um acordo furado....aí espera em novembro, chega em novembro e Rio e Minas pioram com a pandemia e fica pra Dezembro",-1
5,"@MauraVelosodeM1 @randolfeap Reflita vc!! Os pobres, miseráveis e invisíveis muitos estavam nos movimentos de ontem … Bolsonaro ajudou muito essas pessoas na pandemia com a ajuda emergencial! Em gratidão, esse povo está apoiando Bolsonaro !",-1
10,@___caio_Ocrente @Luclaw1 @JornalismoWando O discurso anti-sus se enfraqueceu na pandemia. Sobraram somente os lixos que ninguém liga. Tá engraçado demais ver que os lixos bolsonaristas estão presos ao método. Fico mais tranquilo ao saber que ele perderá feio as eleições em 2022.,-1
11,@fernandesjoabe5 @joaoguilherm @ESPNBrasil E na 23° rodada a pandemia vai ter acabado ?,-2
12,"@alvarodias_ ""O Podemos descarta aderir ao movimento de impeachment do Presidente JB, por entender que a abertura de uma nova crise política, em meio à pandemia do coronavírus, desemprego e crise econômica, só agravaria o sofrimento o sofrimento das camadas mais vulneráveis.."" Que Vergonha❗",-2
13,tô presa no meu antes x depois do início da pandemia pra cá (perdi 10 kgs),-2
14,2019 eu tava muito feia então o sarrafo da pandemia não tá tão alto (silver lining),-1
15,previsão: a pandemia vai acabar na rodada 23 do brasileirão https://t.co/1j1Qw2aRFg,-1
24,"Se trabalha pela torcida, faz uma enquete sobre a opinião da torcida em relação a postura da diretoria no quesito público e demais problemas relacionados a pandemia.",-4
26,@TNTSportsBR O correto seria condições iguais. Mas em nada estão preocupados com pandemia.,-1
